In [1]:
import os
import json
import random
from datetime import datetime
import tempfile
import base64
from pathlib import Path
from dotenv import load_dotenv

from cosmosdb_utils import CosmosDBManager
from request_log import RequestLog

from ai_ocr.chains import get_structured_data, get_final_reasoning_from_markdown
from ai_ocr.process import process_pdf

## Static output schema - demo
### Load up demo assets and call the Doc Intell and subsequentially GPT-4T

In [8]:
system_prompt =  ''
with open('../demo/medical/system_prompt', 'r') as file_sys_prompt:
    system_prompt = file_sys_prompt.read()

output_schema = ''
with open('../demo/medical/surgery_output_schema.json', 'r') as file_output_schema:
    output_schema = file_output_schema.read()

input_path = '../demo/medical/eyes_surgery_pre_1_4.pdf'

ocr_response = process_pdf(file_to_ocr=input_path,
                        prompt=system_prompt,
                        json_schema=output_schema)
print(f'OCR Response: {ocr_response}')
            
#nl_reponse = get_final_reasoning_from_markdown(ocr_response)
#print(f"NL Response: {nl_reponse}")

/home/aga/azureai/azure-doc-extraction-gbb-ai/backend/temp/imgs/page_1_img_1.jpeg
/home/aga/azureai/azure-doc-extraction-gbb-ai/backend/temp/imgs/page_2_img_1.jpeg
OCR Response: {"id": "medical_report", "categorization": "medical_report", "title": "Medical Report", "type": "object", "properties": {"doctor": {"type": "object", "properties": {"specialty": "Oculistica", "name": "Dottore di Ricerca Responsabile Chirurgia Oculistica Clinica Rugani", "clinic": "Clinica Rugani - Siena", "phone": "335-8118324", "fax": "0577-270652"}}, "patient": {"type": "object", "properties": {"name": "Gallo Alberto"}}, "post_surgery_follow_up": {"type": "array", "items": [{"type": "object", "properties": {"period": "1 SETTIMANA", "date": "2002-12-06", "ODv": "Berre", "ODT": "", "OSv": "8/10", "OST": "", "therapy": "Fluoton x4"}}, {"type": "object", "properties": {"period": "1 MESE", "date": "2003-01-21", "ODv": "9/10", "ODT": "17", "OSv": "10/10", "OST": "17", "therapy": "Fluoton 4 volte al d\u00ec"}}, {"ty

## Dynamic output schema - demo

### Step by step approach
1. extract OCR with Doc Intell  
2. Use GPT-4T to determine the category of the document  
3. Load the corresponding json_schema_output from db where categoryId = category determined  
4. Use GPT-4T to apply the schema to the markdown results of the OCR  
5. (optional) Use GPT-4T to provde a natural language summary of the results  

### 1. & 2. Extract + Classification

In [3]:
from ai_ocr.azure.doc_intelligence import get_ocr_results
from ai_ocr.chains import classify_doc_with_llm

input_path = '../demo/medical/eyes_surgery_pre_1_4.pdf'

#1. extract OCR with doc intell
ocr_result = get_ocr_results(input_path)
#print(ocr_result.content)

#2. classify with LLM
classification_sytem_prompt = ''
with open('../demo/medical/classification_system_prompt', 'r') as class_sys_prompt:
    classification_sytem_prompt = class_sys_prompt.read()

category = classify_doc_with_llm(ocr_result.content, classification_sytem_prompt)
print(category.content)

medical_report


### 3. Load output schema for the right category from CosmosDB

In [10]:
from cosmosdb_utils import CosmosDBManager

db = CosmosDBManager()
output_schema = db.read_schema(category.content)

if output_schema == 'None':
    print(f'Schema not found for category: {category.content}')
else:
    print(output_schema['title'])

Medical Report


### 4. Use GPT-4T to apply the schema on the OCR results

In [11]:
system_prompt =  ''
with open('../demo/medical/system_prompt', 'r') as file_sys_prompt:
    system_prompt = file_sys_prompt.read()


ocr_response = process_pdf(file_to_ocr=input_path,
                        prompt=system_prompt,
                        json_schema=json.dumps(output_schema))

print(f'OCR Response: {ocr_response}')
            


/home/aga/azureai/azure-doc-extraction-gbb-ai/backend/temp/imgs/page_1_img_1.jpeg
/home/aga/azureai/azure-doc-extraction-gbb-ai/backend/temp/imgs/page_2_img_1.jpeg
OCR Response: {"id": "medical_report", "categorization": "", "title": "Medical Report", "type": "object", "properties": {"doctor": {"type": "object", "properties": {"specialty": "Oculistica", "name": "Dottore di Ricerca Responsabile Chirurgia Oculistica Clinica Rugani", "clinic": "Clinica Rugani - Siena", "phone": "335-8118324", "fax": "0577-270652"}}, "patient": {"type": "object", "properties": {"name": "Gallo Alberto"}}, "post_surgery_follow_up": [{"period": "1 SETTIMANA", "date": "2002-12-06", "ODv": "Berre", "ODT": "", "OSv": "8/10", "OST": "", "therapy": "Fluoton x4"}, {"period": "1 MESE", "date": "2003-01-21", "ODv": "9/10", "ODT": "17", "OSv": "10/10", "OST": "17", "therapy": "Fluoton"}, {"period": "2-3 MESI", "date": "2003-03-18", "ODv": "10/10", "ODT": "18", "OSv": "10/10", "OST": "18", "therapy": "Fluoton x3 volte"

In [12]:
with open('output.json', 'w') as f:
    f.write(ocr_response)